<u><h1 style="color:#21B6A8"> Exercice : Dataset cleaning and mapinulation </h1></u>

<h2 style="color:#98D7C2">Importing library 🐼</h2>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

<h2 style="color:#98D7C2">Reading datasets</h2>

In [ ]:
customers = pd.read_csv('dataset/customers.csv')
products = pd.read_csv('dataset/products.csv')
transactions = pd.read_csv('dataset/transactions.csv')

<h2 style="color:#98D7C2">Copy creations</h2>

In [ ]:
customers_copie = customers.copy()
products_copie = products.copy()
transactions_copie = transactions.copy()

<h2 style="color:#98D7C2">Displaying data </h2>

In [ ]:
customers_copie.head()
products_copie(20) 
transactions_copie.head()

<h2 style="color:#98D7C2">Data description</h2>

In [ ]:
transactions_copie.describe()

<i style="color:#90ADC6"> -> ici, je vois que la valeur "test" revient trop souvent dans la colonne date, ce n’est pas une date valide, pandas considère toute la colonne comme un objet. Après avoir analysé les lignes avec cette valeur, je décide de "supprimer" ou plutôt de filtrer ces lignes car ca ressemble à des valeurs de test. </i>

<h2 style="color:#98D7C2">Deleting ‘test’ values</h2>

In [ ]:
transactions_copie = transactions_copie[~transactions_copie['date'].astype(str).str.contains("test", case=False, na=False)]

<h2 style="color:#98D7C2">Checking data types</h2>

In [ ]:
transactions_copie.dtypes

<i style="color:#90ADC6"> -> ici, je m'aperçois que la date est au format objet, ce n'est pas correct, je vais donc la convertir en datetime. </i>

<h2 style="color:#98D7C2">Converting date to date</h2>

In [ ]:
transactions_copie.loc[:, 'date'] = pd.to_datetime(transactions_copie['date'])

<h2 style="color:#98D7C2">Checking for duplicates</h2>

In [ ]:
transactions_copie.duplicated().sum()
products_copie.duplicated().sum()
customers_copie.duplicated().sum()

<h2 style="color:#98D7C2">Checking for null values</h2>

In [ ]:
transactions_copie.isnull().sum()
products_copie.isnull().sum()
customers_copie.isnull().sum()

<h2 style="color:#98D7C2">Correction negative values</h2>

In [ ]:
products_copie.describe()

In [ ]:
products_copie['price'].min()

In [ ]:
products_copie = products_copie[products_copie['price']>=0]

<h2 style="color:#98D7C2">Display monthly revenue trend</h2>

In [ ]:
merged = pd.merge(transactions_copie, products_copie[['price', 'id_prod']], on='id_prod')
merged['month'] = merged['date'].dt.to_period('M')
ca_mois = merged.groupby('month')['price'].sum().reset_index()
ca_mois

<i style="color:#90ADC6"> -> Ici, je choisis de faire une jointure interne car je souhaite uniquement conserver les transactions pour lesquelles le produit existe dans la table des produits. </i>

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(ca_mois['month'].astype(str), ca_mois['price'], marker='x', linestyle='-')
plt.title("Chiffre d'affaires par mois")
plt.xlabel("Mois")
plt.ylabel("Chiffre d'affaires (€)")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

<h2 style="color:#98D7C2">Calculates the number of transactions per customer</h2>

In [ ]:
merge_trans_client = pd.merge(transactions_copie, customers_copie, on='client_id')
merge_trans_client.groupby('client_id')['client_id'].count().reset_index(name='nmb_of_transactions')

In [ ]:
merge_trans_client[merge_trans_client['nmb_of_transactions'] > 1000]

<i style="color:#90ADC6"> -> Ici, je fais une jointure interne car je veux afficher seulement les clients qui ont des transactions. Sans la jointure, je peux afficher 1 seule transaction par client or que moi je veux toutes les transactions qu'un client a fait donc la jointure est nécessaire ici.  </i>